In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

bttai_ajl_2025_path = kagglehub.competition_download('bttai-ajl-2025')

print('Data source import complete.')


In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# A Simple Starter Code for the AJL Competition

@Cindy Deng

---


Hi! This starter code is designed to help you get familiar with basic Kaggle operations and guide you through the basic workflow of a machine learning project.

The code outlines essential steps including data loading, preprocessing, model building, training, and generating predictions. Each section serves as a foundation, but there are many ways to enhance each step to improve your final model's accuracy. Feel free to experiment with different data augmentation techniques, model architectures, and tuning methods to optimize your final results! Some amazing tutorials are available through your AI Studio course in Canvas / in the 'Resource' section of this Kaggle competition.

Good luck and have fun!

---

## Note - About file path

You could use the cell above to print the names of the file directories and get the following directories:

```
/kaggle/input/bttai-ajl-2025/sample_submission.csv
/kaggle/input/bttai-ajl-2025/train.csv
/kaggle/input/bttai-ajl-2025/test.csv
/kaggle/input/bttai-ajl-2025/test/test/e0374ae6c1362ff183cfba28ded5421b.jpg
/kaggle/input/bttai-ajl-2025/test/test/437159c605260bdd079af230566af291.jpg
...
...
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/11271bdf2598afdd4260db3125e1f6a5.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/732819951dcf2b53d15ea7b8bb123b71.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/6dcc7a8abb5e1c6e670101f4b6231246.jpg
/kaggle/input/bttai-ajl-2025/train/train/dermatomyositis/e63c3b3f0ab8905e204fe467cc7411f9.jpg
...
...
```



## 1. Import Necessary Libraries

In [ ]:
# 1. Import Necessary Libraries
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# Explanation:
# - pandas and numpy: for data manipulation
# - sklearn: for splitting data and encoding labels
# - tensorflow.keras: for building and training the neural network

## 2. Load Data

Make sure to verify the file paths if you're running on a different platform.

In [ ]:
# 2. Load Data
train_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-ajl-2025/test.csv')

# Add .jpg extension to md5hash column to reference the file_name
train_df['md5hash'] = train_df['md5hash'].astype(str) + '.jpg'
test_df['md5hash'] = test_df['md5hash'].astype(str) + '.jpg'

# Combine label and md5hash to form the correct path
train_df['file_path'] = train_df['label'] + '/' + train_df['md5hash']
test_df['file_path'] = train_df['label'] + '/' + train_df['md5hash']

In [ ]:
# Check the first few rows to understand the structure
train_df.head()

In [ ]:
# Check the first few rows to understand the structure
test_df.head()

## 3. Data Preprocessing


This section demonstrates basic preprocessing techniques. To enhance data quality and model performance, consider incorporating more advanced preprocessing methods.

For further guidance, feel free to take a look at the [Image Preprocessing tutorial](https://colab.research.google.com/drive/1-ItNcRMbZBE6BCwPT-wD8m3YmHqwHxme?usp=sharing)  available in the 'Resources' section of this Kaggle competition.


In [ ]:
# 3. Data Preprocessing
# Encode the labels
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])
train_df['encoded_label'] = train_df['label'].astype(str)

# Split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Define image data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Define the directory paths
train_dir = '/kaggle/input/bttai-ajl-2025/train/train/'

In [ ]:
def create_generator(dataframe, directory, batch_size=32, target_size=(128, 128)):
    """
    Template function to create image generators.
    Students should complete this function to load images and labels properly.
    """
    # Fill in the correct flow_from_dataframe parameters
    generator = train_datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col='file_path',  # Use combined path
        y_col='encoded_label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        validate_filenames=False  # Disable strict filename validation
    )
    return generator

In [ ]:
# Create generators
train_generator = create_generator(train_data, train_dir)
val_generator = create_generator(val_data, train_dir)

In [ ]:
x_batch, y_batch = next(train_generator)

print("Shape of batch images:", x_batch.shape)  # Expected: (batch_size, 128, 128, 3)
print("Shape of batch labels:", y_batch.shape)  # Expected: (batch_size, 16)
print("First label sample:", y_batch[0])  # Should look like [0, 0, 1, 0, ..., 0] (one-hot)


In [ ]:
# List all image file paths in the DataFrame
missing_files = [f for f in train_data['file_path'] if not os.path.exists(os.path.join(train_dir, f))]
print(f"Missing Train Images: {len(missing_files)}")

missing_files_val = [f for f in val_data['file_path'] if not os.path.exists(os.path.join(train_dir, f))]
print(f"Missing Validation Images: {len(missing_files_val)}")

## 4. Build the model


In [ ]:
# Implement the model architecture here.
# Feel free to explore different model types that best serve your purpose.
# Build CNN model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    # add dropout
    Dropout(0.5), # increase from 0.3 - 0.5
    Flatten(),
    # add
    Activation('relu'),
    Dense(128, activation='relu'),
    # add
    Dropout(0.2),
    # Dropout(0.8), # dropping neutrons randomly to prevent overfitting but in this location, causes model to be more inaccurate
    Dense(21, activation='softmax')
])


# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# View model summary
model.summary()

## 5. Train the Model


In [ ]:
# Train your model here.
history = model.fit(train_generator, validation_data=val_generator, epochs=20)




## 6. Make Predictions on Test Data

In [ ]:
# 6. Make Predictions on Test Data
def preprocess_test_data(test_df, directory):
    """
    Template for loading and preprocessing test images.
    """

    test_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values

    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=directory,
        x_col='file_path',  # Column that contains image file names
        target_size=(126,126),
        batch_size=21,
        class_mode=None,  # No labels for test set
        shuffle=False  # Ensure order is maintained for predictions
    )

    return test_generator


In [ ]:
# Load test data
test_dir = '/kaggle/input/bttai-ajl-2025/test/test/'
test_df['file_path'] = test_df['md5hash']

test_generator = preprocess_test_data(test_df, test_dir)

## 7. Generate Predictions

In [ ]:
# Generate predictions based on the trained model
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Then, save the predictions into a CSV file for submission
# Convert class indices to labels
class_labels = {v: k for k, v in train_generator.class_indices.items()}  # Reverse mapping
test_df['predicted_label'] = test_df.index.to_series().apply(lambda i: class_labels[predicted_classes[i]])


In [ ]:
# Remove .jpg from md5hash before saving
test_df['md5hash'] = test_df['md5hash'].str.replace('.jpg', '', regex=False)

#save to csv
submission_file = "skin_condition_predictions.csv"
test_df[['md5hash', 'predicted_label']].to_csv(submission_file, index=False)

print(f"Predictions saved to {submission_file}")



In [ ]:
readSubFile = pd.read_csv(submission_file)

print(readSubFile.head())